<h1 style="background: linear-gradient(180deg,rgb(92, 0, 128) 0%,rgb(46, 0, 153) 75%, rgb(65, 0, 230) 100%); color: white; font-family: 'Raleway', sans-serif; padding: 10px 20px; border-radius: 10px; text-align: center; font-weight:500;">
Modelado con ML
</h1>
<br>

**PRESENTA** Armando Islas

In [3]:
import pickle
import os
import pandas as pd
import numpy as np
import mlflow
import time
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score, accuracy_score, precision_score, recall_score

In [4]:
# Directorio donde se encuentran los archivos pkl
DATA_DIR = "../outputs/text-reps/"

In [5]:
# Definir los modelos a utilizar
models = {
    "LogisticRegression": LogisticRegression(max_iter=200),
    "RandomForestClassifier": RandomForestClassifier(),
    "XGBClassifier": XGBClassifier()
}

# Definir las métricas a calcular
scoring = {
    'f1_macro': make_scorer(f1_score, average='macro'),
    'accuracy': make_scorer(accuracy_score),
    'precision_macro': make_scorer(precision_score, average='macro'),
    'recall_macro': make_scorer(recall_score, average='macro')
}

In [6]:
# Función para cargar los datos
def load_data(file_path):
    with open(file_path, 'rb') as f:
        result = pickle.load(f)
    
    # Extraer X e y del diccionario
    X = result['X']
    y = result['y']
    
    # Podemos extraer la configuración para registro en MLflow
    config = result['config']
    
    return X, y, config

In [7]:
# Función para aplicar LabelEncoder a las etiquetas
def encode_labels(y):
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    # Guardar el mapeo de clases para referencia
    class_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
    return y_encoded, label_encoder, class_mapping

In [9]:
# Función para aplicar oversampling
def apply_oversampling(X, y):
    oversample = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = oversample.fit_resample(X, y)
    return X_resampled, y_resampled

In [10]:
# Función para entrenar y evaluar modelos con validación cruzada
def train_and_evaluate(X, y, model_name, model, dataset_name, config, class_mapping):
    # Configurar la validación cruzada estratificada
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Iniciar MLflow run
    with mlflow.start_run(run_name=f"{model_name}_{dataset_name}"):
        # Registrar el nombre del modelo y del dataset
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("dataset_name", dataset_name)
        
        # Registrar la configuración del dataset
        mlflow.log_param("ngram_range", str(config['ngram_range']))
        mlflow.log_param("mode", config['mode'])
        mlflow.log_param("svd_components", config['svd_components'])
        
        # Registrar el mapeo de clases
        for original_class, encoded_value in class_mapping.items():
            mlflow.log_param(f"class_{encoded_value}", original_class)
        
        # Medir el tiempo de ejecución
        start_time = time.time()
        
        # Realizar validación cruzada con múltiples métricas
        cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring)
        
        # Calcular tiempo total
        exec_time = time.time() - start_time
        
        # Registrar métricas promedio
        for metric_name, metric_values in cv_results.items():
            if metric_name.startswith('test_'):
                # Solo registrar métricas de evaluación (no tiempos)
                metric_avg = np.mean(metric_values)
                mlflow.log_metric(metric_name.replace('test_', ''), metric_avg)
        
        # Registrar tiempo de ejecución
        mlflow.log_metric("execution_time", exec_time)
        
        # Devolver resultados para el dataframe final
        return {
            'dataset': dataset_name,
            'model': model_name,
            'ngram_range': str(config['ngram_range']),
            'mode': config['mode'],
            'svd_components': config['svd_components'],
            'num_classes': len(class_mapping),
            'f1_macro': np.mean(cv_results['test_f1_macro']),
            'accuracy': np.mean(cv_results['test_accuracy']),
            'precision_macro': np.mean(cv_results['test_precision_macro']),
            'recall_macro': np.mean(cv_results['test_recall_macro']),
            'execution_time': exec_time
        }


In [11]:
# Lista para almacenar todos los resultados
all_results = []

# Obtener lista de archivos pkl
pkl_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.pkl')]

print(f"Encontrados {len(pkl_files)} archivos pkl para procesar.")

# Procesar cada archivo
for i, pkl_file in enumerate(pkl_files):
    print(f"Procesando archivo {i+1}/{len(pkl_files)}: {pkl_file}")
    
    # Cargar datos
    X, y, config = load_data(os.path.join(DATA_DIR, pkl_file))
    
    # Aplicar LabelEncoder a las etiquetas
    y_encoded, label_encoder, class_mapping = encode_labels(y)
    print(f"  Clases originales: {label_encoder.classes_}")
    print(f"  Mapeo de clases: {class_mapping}")
    
    # Aplicar oversampling
    X_resampled, y_resampled = apply_oversampling(X, y_encoded)
    
    print(f"  Datos cargados, etiquetas codificadas y balanceadas. Shape: {X_resampled.shape}")
    print(f"  Configuración: n-grams={config['ngram_range']}, mode={config['mode']}, svd_components={config['svd_components']}")
    print(f"  Distribución de clases después de oversampling: {np.bincount(y_resampled)}")
    
    # Entrenar y evaluar cada modelo
    for model_name, model in models.items():
        print(f"  Entrenando {model_name}...")
        
        # Entrenar y evaluar modelo
        result = train_and_evaluate(X_resampled, y_resampled, model_name, model, pkl_file, config, class_mapping)
        
        # Añadir resultados
        all_results.append(result)
        
        print(f"  {model_name} completado. F1-macro: {result['f1_macro']:.4f}")

# Crear dataframe con todos los resultados
results_df = pd.DataFrame(all_results)

# Ordenar por f1_macro descendente
results_df = results_df.sort_values('f1_macro', ascending=False)

# Guardar resultados en CSV
results_df.to_csv('ml_experiment_results.csv', index=False)

# Mostrar los 10 mejores resultados
print("\nTop 10 mejores combinaciones (por F1-macro):")
print(results_df.head(10).to_string(index=False))

# Mostrar análisis agrupado para tener visión general de rendimiento
print("\nRendimiento promedio por tipo de modelo:")
model_perf = results_df.groupby('model')['f1_macro'].mean().sort_values(ascending=False)
print(model_perf)

print("\nRendimiento promedio por n-gramas:")
ngram_perf = results_df.groupby('ngram_range')['f1_macro'].mean().sort_values(ascending=False)
print(ngram_perf)

print("\nRendimiento promedio por modo de vectorización:")
mode_perf = results_df.groupby('mode')['f1_macro'].mean().sort_values(ascending=False)
print(mode_perf)

print(results_df)


Encontrados 27 archivos pkl para procesar.
Procesando archivo 1/27: normalized_all_uni_freq_svd50.pkl
  Clases originales: ['Clickbait' 'No']
  Mapeo de clases: {'Clickbait': 0, 'No': 1}
  Datos cargados, etiquetas codificadas y balanceadas. Shape: (4004, 50)
  Configuración: n-grams=(1, 1), mode=freq, svd_components=50
  Distribución de clases después de oversampling: [2002 2002]
  Entrenando LogisticRegression...
  LogisticRegression completado. F1-macro: 0.6686
  Entrenando RandomForestClassifier...
  RandomForestClassifier completado. F1-macro: 0.8953
  Entrenando XGBClassifier...
  XGBClassifier completado. F1-macro: 0.8808
Procesando archivo 2/27: normalized_token_stop_lemma_uni_freq_svd50.pkl
  Clases originales: ['Clickbait' 'No']
  Mapeo de clases: {'Clickbait': 0, 'No': 1}
  Datos cargados, etiquetas codificadas y balanceadas. Shape: (4004, 50)
  Configuración: n-grams=(1, 1), mode=freq, svd_components=50
  Distribución de clases después de oversampling: [2002 2002]
  Entrena